In [2]:
from scipy.spatial.distance import squareform, pdist
pdist()

TypeError: pdist() missing 1 required positional argument: 'X'

In [ ]:
import numpy as np
import math
import time
import pyaudio
import wave
from scipy.io.wavfile import read
from python_speech_features import mfcc
import os

os.chdir("C:/Users/Administrator/Documents/WeChat Files/zhjadsf/Files/voice/voice")
start = time.clock()

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
RECORD_SECONDS = 2
WAVE_OUTPUT_FILENAME = "output.wav"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

(rate,test1) = read("output.wav")
#test1 = test1[:,0]
(rate,test2) = read("test2.wav")
test2 = test2[:,0]
(rate,test3) = read("test3.wav")
test3 = test3[:,0]
(rate,ref1) = read("ref1.wav")
ref1 = ref1[:,0]
(rate,ref2) = read("ref2.wav")
ref2 = ref2[:,0]
(rate,ref3) = read("ref3.wav")
ref3 = ref3[:,0]

def enframe(x): #分帧
    FrameLen = 240
    FrameInc = 80
    m = math.ceil(len(x)/FrameInc)-3
    v = np.zeros([m,FrameLen])
    for i in range(m):
        v[i] = x[i*80:i*80+240]
    return v

def vad(x): #端点检测
    x = x/max(abs(x))
    FrameInc = 80
    amp1 = 20
    amp2 = 2
    zcr1 = 10
    zcr2 = 5
    maxsilence = 3
    minlen = 15
    status = 0
    count = 0
    silence = 0
    #计算过零率
    tmp1 = enframe(x[0:-2])
    tmp2 = enframe(x[1:-1])
    signs = tmp1*tmp2
    [r,c] = signs.shape
    for i in range(r):
        for j in range(c):
            if signs[i,j]<0:
                signs[i,j]=1
            else:
                signs[i,j]=0
    diffs = tmp1-tmp2
    [r,c] = diffs.shape
    for i in range(r):
        for j in range(c):
            if diffs[i,j]>0.2: #设定好的阈值
                diffs[i,j]=1
            else:
                diffs[i,j]=0
    smultid = signs*diffs
    zcr = smultid.sum(axis=1)
    #计算短时能量
    y = np.zeros([len(x)])
    for i in range(len(x)): #滤波
        y[i] = x[i]-0.9375*x[i-1]
    energy = abs(enframe(y))
    amp = energy.sum(axis=1)
    #调节能量门限
    amp1 = min(amp1, max(amp)/4)
    amp2 = min(amp2, max(amp)/8)
    #开始端点检测
    x1 = 0
    x2 = 0
    n = 0
    while n<len(zcr):
        if status==0 or status==1: #0=静音，1=可能开始
            if amp[n]>amp1: #确信进入语音段
                x1 = max(n-count,1)
                status=2
                silence=0
                count=count+1
            elif amp[n]>amp2 or zcr[n]>zcr2: #可能进入语音段
                status=1
                count=count+1
            else: #静音状态
                status=0
                count=0
        if status==2: #语音段
            if amp[n]>amp2 or zcr[n]>zcr2: #保持在语音段
                count=count+1
            else: #语音将结束
                silence=silence+1
                if silence<maxsilence: #静音不够长，尚未结束
                    count=count+1
                elif count<minlen: #语音长度太短，认为是噪声
                    status=0
                    silence=0
                    count=0
                else:
                    status=3
        if status==3: #语音结束
            break
        n=n+1
    count = count-silence//2-1
    x2 = x1+count-1
    return x[x1*FrameInc:x2*FrameInc]      #,x1*FrameInc,x2*FrameInc

def dtw(t, r):
    t = vad(t)
    r = vad(r)
    t = mfcc(t,rate)
    r = mfcc(r,rate)
    #print(t.shape,r.shape)
    n,o = t.shape
    m,o = r.shape
    l = o
    d = np.zeros([n, m])

    for i in range(n):
        for j in range(m):
            for k in range(l):
                dis = (t[i,k]-r[j,k])*(t[i,k]-r[j,k])
            d[i,j] = d[i,j] + dis
    #print(d)
    D = np.ones((n,m))*1000
    D[0, 0] = d[0, 0]
    for i in range(n):
        for j in range(m):
            if i == 0:
                continue
            else:
                D1 = D[i - 1, j]
                if j > 0:
                    D2 = D[i - 1, j - 1]
                else:
                    D2 = 1000
                if j > 1:
                    D3 = D[i - 1, j - 2]
                else:
                    D3 = 1000
                D[i, j] = d[i, j] + min(D1, D2, D3)
    dist = (D[n - 1, m - 1])
    #print(D)
    return dist

dist1 = dtw(test1, ref1)
dist2 = dtw(test1, ref2)
dist3 = dtw(test1, ref3)
print(dist1,dist2,dist3)
m = min(dist1, dist2, dist3)
if m==dist1:
    print('output is ref1')
elif m ==dist2:
    print('output is ref2')
else:
    print('output is ref3')

end = time.clock()
print(end-start)


In [ ]:
# np.arctan(0.707)
np.arctan2([1,2],[0,1])